# Préparation des données pour le classificateur de prunes africaines

Ce notebook utilise les fonctions existantes dans le dépôt pour préparer les données d'entraînement et de test du modèle de classification des prunes africaines en utilisant le jeu de données Kaggle "African Plums Dataset".

## 1. Configuration de l'environnement pour Google Colab

Commençons par cloner le dépôt GitHub et configurer l'environnement.

In [ ]:
# Vérifier si nous sommes dans Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print(f"Exécution dans Google Colab: {IN_COLAB}")

if IN_COLAB:
    # Cloner le dépôt GitHub
    !git clone https://github.com/CodeStorm-mbe/african-plums-classifier.git
    %cd african-plums-classifier
    
    # Installer les dépendances requises
    !pip install -r requirements.txt
    !pip install kaggle

## 2. Monter Google Drive pour la persistance des données

Pour conserver les données entre les différents notebooks, nous allons utiliser Google Drive comme stockage persistant.

In [ ]:
# Monter Google Drive si nous sommes dans Colab
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Créer un répertoire pour notre projet dans Google Drive
    DRIVE_PROJECT_DIR = "/content/drive/MyDrive/african-plums-classifier"
    DRIVE_DATA_DIR = f"{DRIVE_PROJECT_DIR}/data"
    DRIVE_KAGGLE_DIR = f"{DRIVE_DATA_DIR}/kaggle"
    DRIVE_RAW_DATA_DIR = f"{DRIVE_DATA_DIR}/raw"
    DRIVE_PLUM_DATA_DIR = f"{DRIVE_RAW_DATA_DIR}/plums"
    DRIVE_NON_PLUM_DATA_DIR = f"{DRIVE_RAW_DATA_DIR}/non_plums"
    
    # Créer les répertoires s'ils n'existent pas
    !mkdir -p {DRIVE_PROJECT_DIR}
    !mkdir -p {DRIVE_DATA_DIR}
    !mkdir -p {DRIVE_KAGGLE_DIR}
    !mkdir -p {DRIVE_RAW_DATA_DIR}
    !mkdir -p {DRIVE_PLUM_DATA_DIR}
    !mkdir -p {DRIVE_NON_PLUM_DATA_DIR}
    !mkdir -p {DRIVE_NON_PLUM_DATA_DIR}/non_plum
    
    print(f"Google Drive monté et répertoires créés dans {DRIVE_PROJECT_DIR}")

In [ ]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random
import zipfile
import shutil

# Ajouter le répertoire courant au chemin pour pouvoir importer nos modules
if IN_COLAB:
    # Dans Colab, nous sommes déjà dans le répertoire du projet
    if "/content/african-plums-classifier" not in sys.path:
        sys.path.append("/content/african-plums-classifier")
else:
    # En local, ajouter le répertoire parent
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

# Importer nos modules personnalisés
from data.data_preprocessing import (
    load_and_prepare_data,
    load_and_prepare_two_stage_data,
    visualize_batch,
    analyze_dataset_distribution
)

# Définir les chemins des données
if IN_COLAB:
    # Utiliser les chemins dans Google Drive pour la persistance
    DATA_ROOT = DRIVE_RAW_DATA_DIR
    KAGGLE_DIR = DRIVE_KAGGLE_DIR
    
    # Créer également des liens symboliques pour faciliter l'accès depuis le code existant
    LOCAL_DATA_ROOT = "data/raw"
    LOCAL_KAGGLE_DIR = "data/kaggle"
    
    # Créer les répertoires locaux s'ils n'existent pas
    !mkdir -p {LOCAL_DATA_ROOT}
    !mkdir -p {LOCAL_KAGGLE_DIR}
    
    # Créer des liens symboliques si nécessaire
    if not os.path.exists(LOCAL_DATA_ROOT) or not os.path.islink(LOCAL_DATA_ROOT):
        !rm -rf {LOCAL_DATA_ROOT}
        !ln -s {DATA_ROOT} {LOCAL_DATA_ROOT}
    
    if not os.path.exists(LOCAL_KAGGLE_DIR) or not os.path.islink(LOCAL_KAGGLE_DIR):
        !rm -rf {LOCAL_KAGGLE_DIR}
        !ln -s {KAGGLE_DIR} {LOCAL_KAGGLE_DIR}
else:
    # En local
    DATA_ROOT = "../data/raw"
    KAGGLE_DIR = "../data/kaggle"

PLUM_DATA_DIR = os.path.join(DATA_ROOT, "plums")  # Sous-dossier pour les prunes
NON_PLUM_DATA_DIR = os.path.join(DATA_ROOT, "non_plums")  # Sous-dossier pour les non-prunes

# Vérifier si les répertoires existent
os.makedirs(DATA_ROOT, exist_ok=True)
os.makedirs(PLUM_DATA_DIR, exist_ok=True)
os.makedirs(NON_PLUM_DATA_DIR, exist_ok=True)
os.makedirs(os.path.join(NON_PLUM_DATA_DIR, "non_plum"), exist_ok=True)
os.makedirs(KAGGLE_DIR, exist_ok=True)

# Définir les paramètres
BATCH_SIZE = 32
IMG_SIZE = 224
NUM_WORKERS = 2 if IN_COLAB else 4  # Réduire le nombre de workers dans Colab
RANDOM_SEED = 42

# Fixer les seeds pour la reproductibilité
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Déterminer le device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de: {device}")

# Afficher les chemins des données
print(f"\nChemins des données:")
print(f"DATA_ROOT: {DATA_ROOT}")
print(f"KAGGLE_DIR: {KAGGLE_DIR}")
print(f"PLUM_DATA_DIR: {PLUM_DATA_DIR}")
print(f"NON_PLUM_DATA_DIR: {NON_PLUM_DATA_DIR}")

## 3. Configuration de l'API Kaggle

Pour télécharger le jeu de données Kaggle, nous devons configurer l'API Kaggle.

In [ ]:
# Configuration de l'API Kaggle
if IN_COLAB:
    from google.colab import files
    
    # Vérifier si le fichier kaggle.json existe déjà dans Google Drive
    KAGGLE_CONFIG_PATH = os.path.expanduser('~/.kaggle/kaggle.json')
    DRIVE_KAGGLE_CONFIG_PATH = f"{DRIVE_PROJECT_DIR}/kaggle.json"
    
    # Vérifier si le fichier kaggle.json existe dans Google Drive
    kaggle_config_in_drive = os.path.exists(DRIVE_KAGGLE_CONFIG_PATH)
    
    # Vérifier si le fichier kaggle.json existe localement
    kaggle_config_exists = os.path.exists(KAGGLE_CONFIG_PATH)
    
    if kaggle_config_in_drive:
        print(f"Fichier kaggle.json trouvé dans Google Drive. Utilisation de ce fichier.")
        # Créer le répertoire .kaggle s'il n'existe pas
        os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
        # Copier le fichier de Google Drive vers le répertoire local
        shutil.copy(DRIVE_KAGGLE_CONFIG_PATH, KAGGLE_CONFIG_PATH)
        # Définir les permissions appropriées
        os.chmod(KAGGLE_CONFIG_PATH, 600)
        print("Fichier kaggle.json configuré avec succès.")
    elif not kaggle_config_exists:
        print("Veuillez télécharger votre fichier kaggle.json pour l'authentification Kaggle.")
        print("Vous pouvez le générer sur https://www.kaggle.com/account dans la section 'API'.")
        
        # Télécharger le fichier kaggle.json
        uploaded = files.upload()
        
        # Créer le répertoire .kaggle s'il n'existe pas
        os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
        
        # Déplacer le fichier kaggle.json vers le répertoire .kaggle
        if 'kaggle.json' in uploaded:
            shutil.move('kaggle.json', KAGGLE_CONFIG_PATH)
            # Définir les permissions appropriées
            os.chmod(KAGGLE_CONFIG_PATH, 600)
            # Sauvegarder également dans Google Drive pour une utilisation future
            shutil.copy(KAGGLE_CONFIG_PATH, DRIVE_KAGGLE_CONFIG_PATH)
            print("Fichier kaggle.json configuré avec succès et sauvegardé dans Google Drive.")
        else:
            print("Erreur: Le fichier kaggle.json n'a pas été téléchargé.")
    else:
        print("Le fichier kaggle.json existe déjà localement.")
        # Sauvegarder également dans Google Drive pour une utilisation future
        shutil.copy(KAGGLE_CONFIG_PATH, DRIVE_KAGGLE_CONFIG_PATH)
        print("Fichier kaggle.json sauvegardé dans Google Drive.")

## 4. Téléchargement et préparation du jeu de données Kaggle

Téléchargeons le jeu de données "African Plums Dataset" de Kaggle et préparons-le pour notre modèle.

In [ ]:
def download_kaggle_dataset(force_download=False):
    """Télécharge le jeu de données Kaggle 'African Plums Dataset'."""
    # Vérifier si le jeu de données a déjà été téléchargé
    dataset_zip = os.path.join(KAGGLE_DIR, 'african-plums-dataset.zip')
    if os.path.exists(dataset_zip) and not force_download:
        print(f"Le jeu de données a déjà été téléchargé dans {dataset_zip}.")
        return dataset_zip
    
    print("Téléchargement du jeu de données Kaggle 'African Plums Dataset'...")
    try:
        # Télécharger le jeu de données
        !kaggle datasets download -d arnaudfadja/african-plums-quality-and-defect-assessment-data -p {KAGGLE_DIR}
        print(f"Jeu de données téléchargé avec succès dans {dataset_zip}.")
        return dataset_zip
    except Exception as e:
        print(f"Erreur lors du téléchargement du jeu de données: {e}")
        return None

def extract_and_organize_dataset(dataset_zip, force_extract=False):
    """Extrait et organise le jeu de données Kaggle pour notre modèle."""
    if not os.path.exists(dataset_zip):
        print(f"Le fichier {dataset_zip} n'existe pas.")
        return False
    
    # Vérifier si les données ont déjà été extraites
    extracted_dir = os.path.join(KAGGLE_DIR, 'extracted')
    if os.path.exists(extracted_dir) and not force_extract:
        print(f"Le jeu de données a déjà été extrait dans {extracted_dir}.")
    else:
        print(f"Extraction du jeu de données...")
        os.makedirs(extracted_dir, exist_ok=True)
        
        # Extraire le fichier zip
        with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
            zip_ref.extractall(extracted_dir)
        
        print(f"Jeu de données extrait avec succès dans {extracted_dir}.")
    
    # Organiser les données pour notre modèle
    print("Organisation des données pour notre modèle...")
    
    # Vérifier la structure du jeu de données extrait
    print("Structure du jeu de données extrait:")
    !find {extracted_dir} -type d | sort
    
    # Mapper les classes du jeu de données Kaggle aux classes de notre modèle
    # Selon la description, les classes sont: bruised, cracked, rotten, spotted, unaffected, unripe
    class_mapping = {
        'bruised': 'bruised',
        'cracked': 'cracked',
        'rotten': 'rotten',
        'spotted': 'spotted',
        'unaffected': 'unaffected',
        'unripe': 'unripe'
    }
    
    # Créer les répertoires pour les classes de prunes
    for cls in class_mapping.values():
        os.makedirs(os.path.join(PLUM_DATA_DIR, cls), exist_ok=True)
    
    # Créer le répertoire pour les non-prunes
    non_plum_dir = os.path.join(NON_PLUM_DATA_DIR, "non_plum")
    os.makedirs(non_plum_dir, exist_ok=True)
    
    # Copier les images dans les répertoires appropriés
    dataset_dir = os.path.join(extracted_dir, 'african_plums_dataset')
    if os.path.exists(dataset_dir):
        # Parcourir les sous-répertoires du jeu de données
        for src_cls, dst_cls in class_mapping.items():
            src_dir = os.path.join(dataset_dir, src_cls)
            dst_dir = os.path.join(PLUM_DATA_DIR, dst_cls)
            
            if os.path.exists(src_dir):
                # Copier les images
                print(f"Copie des images de {src_dir} vers {dst_dir}...")
                !cp -r {src_dir}/* {dst_dir}/
            else:
                print(f"Le répertoire {src_dir} n'existe pas.")
        
        print("Images copiées avec succès.")
        return True
    else:
        print(f"Le répertoire {dataset_dir} n'existe pas.")
        return False

# Télécharger et préparer le jeu de données
dataset_zip = download_kaggle_dataset(force_download=False)
if dataset_zip:
    success = extract_and_organize_dataset(dataset_zip, force_extract=False)
    if success:
        print("Jeu de données Kaggle préparé avec succès pour notre modèle.")
    else:
        print("Erreur lors de la préparation du jeu de données Kaggle.")

## 5. Création d'images non-prunes

Le jeu de données Kaggle ne contient que des images de prunes. Pour notre modèle à deux étapes, nous avons également besoin d'images non-prunes. Nous allons télécharger quelques images non-prunes à partir d'un autre jeu de données.

In [ ]:
def download_non_plum_images(num_images=100):
    """Télécharge des images non-prunes à partir d'un autre jeu de données Kaggle."""
    non_plum_dir = os.path.join(NON_PLUM_DATA_DIR, "non_plum")
    
    # Vérifier si des images non-prunes existent déjà
    existing_images = [f for f in os.listdir(non_plum_dir) if os.path.isfile(os.path.join(non_plum_dir, f))]
    if existing_images:
        print(f"Des images non-prunes existent déjà ({len(existing_images)} images).")
        return
    
    print("Téléchargement d'images non-prunes...")
    
    # Option 1: Télécharger des images de fruits (autres que des prunes) à partir d'un jeu de données Kaggle
    try:
        # Télécharger un jeu de données de fruits
        !kaggle datasets download -d moltean/fruits -p {KAGGLE_DIR}
        
        # Extraire le jeu de données
        fruits_zip = os.path.join(KAGGLE_DIR, 'fruits.zip')
        fruits_dir = os.path.join(KAGGLE_DIR, 'fruits')
        os.makedirs(fruits_dir, exist_ok=True)
        
        with zipfile.ZipFile(fruits_zip, 'r') as zip_ref:
            zip_ref.extractall(fruits_dir)
        
        # Sélectionner des images aléatoires (excluant les prunes)
        import glob
        all_fruit_images = []
        for fruit_dir in glob.glob(os.path.join(fruits_dir, 'fruits-360/Training/*')):
            fruit_name = os.path.basename(fruit_dir).lower()
            if 'plum' not in fruit_name and 'prune' not in fruit_name:
                all_fruit_images.extend(glob.glob(os.path.join(fruit_dir, '*.jpg')))
        
        # Sélectionner un sous-ensemble aléatoire
        if all_fruit_images:
            selected_images = random.sample(all_fruit_images, min(num_images, len(all_fruit_images)))
            
            # Copier les images sélectionnées
            for i, img_path in enumerate(selected_images):
                dst_path = os.path.join(non_plum_dir, f"non_plum_{i+1}.jpg")
                shutil.copy(img_path, dst_path)
            
            print(f"{len(selected_images)} images non-prunes copiées avec succès.")
        else:
            print("Aucune image de fruit trouvée.")
            return False
        
        return True
    except Exception as e:
        print(f"Erreur lors du téléchargement d'images non-prunes: {e}")
        
        # Option 2: Créer des images synthétiques si le téléchargement échoue
        print("Création d'images non-prunes synthétiques...")
        
        for i in range(num_images):
            # Couleur aléatoire qui n'est pas proche des couleurs de prune
            color = (random.randint(0, 100), random.randint(150, 255), random.randint(150, 255))
            
            # Créer une image
            img = Image.new('RGB', (224, 224), (255, 255, 255))
            pixels = img.load()
            
            # Dessiner une forme aléatoire (carré ou triangle)
            shape = random.choice(['square', 'triangle'])
            
            if shape == 'square':
                # Dessiner un carré
                size = random.randint(100, 150)
                top_left = (random.randint(0, 224-size), random.randint(0, 224-size))
                
                for x in range(top_left[0], top_left[0] + size):
                    for y in range(top_left[1], top_left[1] + size):
                        if 0 <= x < 224 and 0 <= y < 224:
                            # Ajouter du bruit à chaque pixel
                            pixel_color = [max(0, min(255, c + random.randint(-10, 10))) for c in color]
                            pixels[x, y] = tuple(pixel_color)
            else:
                # Dessiner un triangle
                p1 = (random.randint(50, 174), random.randint(50, 174))
                p2 = (p1[0] + random.randint(30, 50), p1[1] + random.randint(30, 50))
                p3 = (p1[0] - random.randint(0, 30), p2[1])
                
                # Remplir le triangle (algorithme simple)
                min_x = min(p1[0], p2[0], p3[0])
                max_x = max(p1[0], p2[0], p3[0])
                min_y = min(p1[1], p2[1], p3[1])
                max_y = max(p1[1], p2[1], p3[1])
                
                for x in range(min_x, max_x + 1):
                    for y in range(min_y, max_y + 1):
                        if 0 <= x < 224 and 0 <= y < 224:
                            # Vérification simple si le point est dans le triangle
                            if (x >= p1[0] and y >= p1[1] and x <= p2[0] and y <= p2[1]):
                                # Ajouter du bruit à chaque pixel
                                pixel_color = [max(0, min(255, c + random.randint(-10, 10))) for c in color]
                                pixels[x, y] = tuple(pixel_color)
            
            # Sauvegarder l'image
            img_path = os.path.join(non_plum_dir, f"non_plum_{i+1}.jpg")
            img.save(img_path)
        
        print(f"{num_images} images non-prunes synthétiques créées avec succès.")
        return True

# Télécharger des images non-prunes
download_non_plum_images(num_images=100)

## 6. Exploration des données

Explorons les données que nous avons préparées.

In [ ]:
# Fonction simple pour explorer les répertoires de données
def explore_directory(directory):
    if not os.path.exists(directory):
        print(f"Le répertoire {directory} n'existe pas.")
        return
    
    print(f"Contenu du répertoire {directory}:")
    subdirs = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    print(f"Sous-dossiers: {subdirs}")
    
    for subdir in subdirs:
        subdir_path = os.path.join(directory, subdir)
        files = [f for f in os.listdir(subdir_path) if os.path.isfile(os.path.join(subdir_path, f))]
        print(f"  - {subdir}: {len(files)} fichiers")

# Explorer les répertoires de données
print("=== Exploration des données de prunes ===")
explore_directory(PLUM_DATA_DIR)

print("\n=== Exploration des données non-prunes ===")
explore_directory(NON_PLUM_DATA_DIR)

## 7. Analyse de la distribution des classes

Utilisons la fonction `analyze_dataset_distribution` du module `data_preprocessing` pour analyser la distribution des classes.

In [ ]:
# Analyser la distribution des classes
try:
    print("Analyse de la distribution des classes de prunes...")
    class_counts, distribution_img = analyze_dataset_distribution(PLUM_DATA_DIR)
    
    # Afficher les résultats
    print("\nDistribution des classes de prunes:")
    for cls, count in class_counts.items():
        print(f"  - {cls}: {count} images")
    
    # Afficher le graphique de distribution
    if os.path.exists('class_distribution.png'):
        plt.figure(figsize=(10, 6))
        img = plt.imread('class_distribution.png')
        plt.imshow(img)
        plt.axis('off')
        plt.title('Distribution des classes')
        plt.show()
        
        # Sauvegarder le graphique dans Google Drive si nous sommes dans Colab
        if IN_COLAB:
            distribution_img_path = f"{DRIVE_PROJECT_DIR}/class_distribution.png"
            shutil.copy('class_distribution.png', distribution_img_path)
            print(f"Graphique de distribution sauvegardé dans Google Drive: {distribution_img_path}")
except Exception as e:
    print(f"Erreur lors de l'analyse de la distribution des classes: {e}")

## 8. Visualisation des exemples d'images

Visualisons quelques exemples d'images de chaque classe.

In [ ]:
def visualize_examples(data_dir, num_examples=3):
    """Visualise des exemples d'images de chaque classe."""
    import glob
    import matplotlib.pyplot as plt
    
    # Obtenir les classes
    classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    
    # Créer une figure avec une ligne par classe
    fig, axes = plt.subplots(len(classes), num_examples, figsize=(15, 3*len(classes)))
    
    # Pour chaque classe
    for i, cls in enumerate(classes):
        # Obtenir les chemins des images
        img_paths = glob.glob(os.path.join(data_dir, cls, '*'))
        
        # Sélectionner des images aléatoires
        selected_paths = random.sample(img_paths, min(num_examples, len(img_paths)))
        
        # Afficher chaque image
        for j, path in enumerate(selected_paths):
            img = Image.open(path)
            if len(classes) > 1:
                axes[i, j].imshow(img)
                axes[i, j].set_title(f"{cls} - {j+1}")
                axes[i, j].axis('off')
            else:
                axes[j].imshow(img)
                axes[j].set_title(f"{cls} - {j+1}")
                axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualiser des exemples de prunes
print("=== Exemples d'images de prunes ===")
visualize_examples(PLUM_DATA_DIR)

# Visualiser des exemples de non-prunes
print("\n=== Exemples d'images non-prunes ===")
visualize_examples(NON_PLUM_DATA_DIR)

## 9. Préparation des données pour le modèle à deux étapes

Utilisons la fonction `load_and_prepare_two_stage_data` du module `data_preprocessing` pour préparer les données.

In [ ]:
# Vérifier si les répertoires de données existent et contiennent des images
def check_data_availability():
    # Vérifier le répertoire des prunes
    plum_classes = [d for d in os.listdir(PLUM_DATA_DIR) if os.path.isdir(os.path.join(PLUM_DATA_DIR, d))]
    if not plum_classes:
        print(f"Aucune classe de prune trouvée dans {PLUM_DATA_DIR}. Veuillez ajouter des données.")
        return False
    
    # Vérifier le répertoire des non-prunes
    non_plum_dir = os.path.join(NON_PLUM_DATA_DIR, "non_plum")
    if not os.path.exists(non_plum_dir):
        print(f"Le répertoire {non_plum_dir} n'existe pas. Veuillez créer ce répertoire et y ajouter des images.")
        return False
    
    return True

# Vérifier la disponibilité des données
data_available = check_data_availability()

if data_available:
    try:
        # Charger et préparer les données pour les deux étapes
        print("Chargement des données pour les deux étapes...")
        (detection_train_loader, detection_val_loader, detection_test_loader, detection_class_names), \
        (classification_train_loader, classification_val_loader, classification_test_loader, classification_class_names) = \
            load_and_prepare_two_stage_data(
                PLUM_DATA_DIR, 
                NON_PLUM_DATA_DIR,
                batch_size=BATCH_SIZE, 
                img_size=IMG_SIZE,
                num_workers=NUM_WORKERS
            )
        
        print(f"Classes de détection: {detection_class_names}")
        print(f"Classes de classification: {classification_class_names}")
        
        # Afficher les tailles des datasets
        print(f"\nTailles des datasets de détection:")
        print(f"  - Entraînement: {len(detection_train_loader.dataset)} images")
        print(f"  - Validation: {len(detection_val_loader.dataset)} images")
        print(f"  - Test: {len(detection_test_loader.dataset)} images")
        
        print(f"\nTailles des datasets de classification:")
        print(f"  - Entraînement: {len(classification_train_loader.dataset)} images")
        print(f"  - Validation: {len(classification_val_loader.dataset)} images")
        print(f"  - Test: {len(classification_test_loader.dataset)} images")
        
        # Sauvegarder les informations des classes dans Google Drive si nous sommes dans Colab
        if IN_COLAB:
            import json
            classes_info = {
                'detection_class_names': detection_class_names,
                'classification_class_names': classification_class_names
            }
            classes_info_path = f"{DRIVE_PROJECT_DIR}/classes_info.json"
            with open(classes_info_path, 'w') as f:
                json.dump(classes_info, f)
            print(f"Informations des classes sauvegardées dans Google Drive: {classes_info_path}")
    except Exception as e:
        print(f"Erreur lors du chargement des données: {e}")
else:
    print("Veuillez d'abord ajouter des données dans les répertoires appropriés.")

## 10. Visualisation d'un lot d'images

Utilisons la fonction `visualize_batch` du module `data_preprocessing` pour visualiser un lot d'images.

In [ ]:
# Visualiser un lot d'images
if data_available and 'detection_train_loader' in locals():
    try:
        print("Visualisation d'un lot d'images de détection...")
        # Obtenir un lot d'images
        images, labels = next(iter(detection_train_loader))
        
        # Visualiser le lot
        visualize_batch(images, labels, detection_class_names)
    except Exception as e:
        print(f"Erreur lors de la visualisation du lot d'images: {e}")

if data_available and 'classification_train_loader' in locals():
    try:
        print("\nVisualisation d'un lot d'images de classification...")
        # Obtenir un lot d'images
        images, labels = next(iter(classification_train_loader))
        
        # Visualiser le lot
        visualize_batch(images, labels, classification_class_names)
    except Exception as e:
        print(f"Erreur lors de la visualisation du lot d'images: {e}")

## 11. Sauvegarde des informations de préparation des données

Sauvegardons les informations importantes pour les notebooks suivants.

In [ ]:
# Sauvegarder les informations importantes dans Google Drive
if IN_COLAB and data_available:
    import json
    
    # Créer un dictionnaire avec les informations importantes
    data_prep_info = {
        'data_root': DATA_ROOT,
        'plum_data_dir': PLUM_DATA_DIR,
        'non_plum_data_dir': NON_PLUM_DATA_DIR,
        'img_size': IMG_SIZE,
        'batch_size': BATCH_SIZE,
        'detection_class_names': detection_class_names if 'detection_class_names' in locals() else None,
        'classification_class_names': classification_class_names if 'classification_class_names' in locals() else None,
        'data_prepared': True,
        'date_prepared': import time; time.strftime("%Y-%m-%d %H:%M:%S")
    }
    
    # Sauvegarder les informations dans Google Drive
    data_prep_info_path = f"{DRIVE_PROJECT_DIR}/data_prep_info.json"
    with open(data_prep_info_path, 'w') as f:
        json.dump(data_prep_info, f, indent=4)
    
    print(f"Informations de préparation des données sauvegardées dans Google Drive: {data_prep_info_path}")
    print("Ces informations seront utilisées par les notebooks suivants.")

## 12. Conclusion

Dans ce notebook, nous avons utilisé les fonctions existantes du module `data_preprocessing` pour :
1. Télécharger et préparer le jeu de données Kaggle "African Plums Dataset"
2. Télécharger des images non-prunes pour compléter notre jeu de données
3. Explorer et analyser la distribution des classes
4. Visualiser des exemples d'images
5. Préparer les données pour notre modèle à deux étapes
6. Sauvegarder les informations importantes dans Google Drive pour les notebooks suivants

Les données sont maintenant prêtes pour l'entraînement du modèle dans le notebook suivant.